In [1]:
from random import normalvariate

from faker import Faker
from texttable import Texttable

faker = Faker()

In [2]:
def print_table(headers, rows):
    table = Texttable()
    table.add_rows([headers] + rows)
    print(table.draw())

In [6]:
class Employee:
    def __init__(self):
        self.name = faker.name()
        self.email = faker.email()
        self.salary = int(max(1000, normalvariate(20000, 20000)))

In [7]:
e = Employee()

e.name, e.email, e.salary

('Kevin Mccoy', 'wlee@hotmail.com', 14542)

In [8]:
with open('./employees.csv', 'w') as f:
    f.write('name,email,salary\n')
    for _ in range(1000):
        e = Employee()
        f.write(f'{e.name},{e.email},{e.salary}\n')

In [9]:
!head -n 10 ./employees.csv

name,email,salary
Andrea Kelly,williamsonmichelle@hotmail.com,1000
Vanessa King,lauren00@yahoo.com,1006
Stacy Patterson,lroberts@edwards-perkins.com,20221
Andrea Weber,xgomez@allen.com,43261
Melissa Owens,eric53@gmail.com,43438
Justin Turner,michelle39@yahoo.com,1000
Matthew Brown,jeremiah85@butler-rocha.biz,15844
David Jones,benjamin18@hotmail.com,31612
Kerry Brennan,dennis93@yahoo.com,9857


## Select all with simple filter

```sql
SELECT 
    * 
FROM 
    employees 
WHERE 
    salary > 2000 
```

In [10]:
with open('./employees.csv', 'r') as f:
    headers = [h for h in f.readline().split(',')]
    
    rows = []
    for raw in f.readlines():
        name, email, salary = raw.split(',')
        
        if int(salary) > 2000:
            rows.append([name, email, salary])
        
print_table(headers, rows)

+--------------------------+-----------------------------------------+--------+
|           name           |                  email                  | salary |
|                          |                                         |        |
+==========================+=========================================+========+
| Stacy Patterson          | lroberts@edwards-perkins.com            | 20221  |
+--------------------------+-----------------------------------------+--------+
| Andrea Weber             | xgomez@allen.com                        | 43261  |
+--------------------------+-----------------------------------------+--------+
| Melissa Owens            | eric53@gmail.com                        | 43438  |
+--------------------------+-----------------------------------------+--------+
| Matthew Brown            | jeremiah85@butler-rocha.biz             | 15844  |
+--------------------------+-----------------------------------------+--------+
| David Jones              | benjamin18@

## Select all with universal filter

```sql
SELECT 
    * 
FROM 
    employees 
WHERE 
    salary > 2000 and salary < 4000
```

In [11]:
def where(condition):
    with open('./employees.csv', 'r') as f:
        headers = f.readline().split(',')

        rows = []
        for raw in f.readlines():
            name, email, salary = raw.split(',')
            
            if condition(dict(name=name, email=email, salary=int(salary))):
                rows.append([name, email, salary])
        
    print_table(headers, rows)
    
where(lambda x: x['salary'] > 2000 and x['salary'] < 4000)    

+------------------+-------------------------------------+--------+
|       name       |                email                | salary |
|                  |                                     |        |
+==================+=====================================+========+
| Jason Rodriguez  | jessicabradley@roberts.info         | 3413   |
+------------------+-------------------------------------+--------+
| Darlene Cole     | rmontoya@byrd.org                   | 2773   |
+------------------+-------------------------------------+--------+
| Hunter Taylor MD | vjohnson@gmail.com                  | 2269   |
+------------------+-------------------------------------+--------+
| Barbara Lambert  | huntercynthia@stephens-melendez.com | 3042   |
+------------------+-------------------------------------+--------+
| Lori Taylor      | tanya41@hotmail.com                 | 3133   |
+------------------+-------------------------------------+--------+
| Barbara Hill     | nmoses@white-murphy.com    

## Complex query

```sql
SELECT 
    name,
    salary
FROM 
    employees 
WHERE 
    name LIKE B%
ORDER BY salary
LIMIT 5
```

In [13]:
(Table('employees.csv')    
    .select('name', 'salary')
    .where(lambda x: x['name'].startswith('B'))
    .order_by('salary', asc=False)
    .limit(5)
    .collect())

+------------------+--------+
|       name       | salary |
+==================+========+
| Betty Hutchinson | 8473   |
+------------------+--------+
| Brian Brown      | 6984   |
+------------------+--------+
| Brian Webb       | 6812   |
+------------------+--------+
| Brian Cox II     | 61925  |
+------------------+--------+
| Brandon Brown    | 61471  |
+------------------+--------+


In [12]:
class Table:
    
    def __init__(self, filepath):
        with open(filepath, 'r') as f:
            self.columns = f.readline().strip().split(',')
            self.rows = []
            
            for line in f.readlines():
                entries = line.strip().split(',')
                
                self.rows.append({
                    column: entries[i]
                    for i, column in enumerate(self.columns)
                })
            
    def select(self, *columns):
        self.columns = columns
        
        return self
    
    def limit(self, limit):
        self.rows = self.rows[:limit]

        return self
    
    def offset(self, offset):
        self.rows = self.rows[offset:]
    
        return self
    
    def order_by(self, column, asc=True):
        self.rows = sorted(self.rows, key=lambda x: x[column], reverse=not asc)
        
        return self
    
    def where(self, condition):
        self.rows = [row for row in self.rows if condition(row)]
        
        return self
    
    def collect(self):        
        print_table(
            self.columns, 
            [[row[column] for column in self.columns] for row in self.rows])